In [1]:
#Blending 的前提是：個別模型的效果都很好，並且模型差異大大
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
df_train = pd.read_csv('ch2_house_train_clean.csv')

In [3]:
df_train_y = df_train['單價(元/平方公尺)']

In [4]:
df_train = df_train.drop(['單價(元/平方公尺)'],axis =1)

In [5]:
display(df_train.head())

,土地移轉總面積(平方公尺),建物現況格局-廳,建物現況格局-房,建物現況格局-衛,建物移轉總面積(平方公尺),總價(元),車位移轉總面積(平方公尺),車位總價(元),num_of_bus_stations_in_100m,income_avg,...,土地區段位置/建物區段門牌,建物型態,建物現況格局-隔間,有無管理組織,車位類別,都市土地使用分區,鄉鎮市區,非都市土地使用分區,location_type,nearest_tarin_station
0,2.364366,-1.567971,-1.419390,-1.495833,-1.353538,-0.910140,-0.539096,-0.389257,-1.429543,0.102731,...,933,2,0,1,1,1,6,1,1,40
1,0.631359,0.810838,-0.732958,-0.343334,-0.343538,-0.661491,-0.539096,-0.389257,1.772410,-0.432691,...,12,1,0,1,1,0,1,1,3,39
2,2.364366,0.810838,1.326338,2.537915,2.297815,2.232775,2.427249,-0.389257,1.132019,0.450769,...,1090,0,0,0,4,0,5,1,2,23
3,0.297232,0.810838,0.639906,0.809166,-0.095638,-0.443924,-0.539096,-0.389257,-0.148762,-0.435469,...,717,1,0,1,1,0,9,1,3,52
4,-0.977402,0.810838,-0.046526,-0.343334,-0.678820,-0.319600,-0.539096,-0.389257,-1.429543,-0.729956,...,55,0,0,0,1,2,0,1,3,82


In [6]:
x_train, x_test, y_train, y_test = train_test_split(df_train,df_train_y,test_size=0.2,random_state=1)

In [7]:
#n_estimators: 也就是弱学习器的最大迭代次数，或者说最大的弱学习器的个数。
#              一般来说n_estimators太小，容易欠拟合，n_estimators太大，
#              又容易过拟合，一般选择一个适中的数值。默认是100。在实际调参的过程中，
#              我们常常将n_estimators和下面介绍的参数learning_rate一起考虑。

# learning_rate: 即每个弱学习器的权重缩减系数ν，也称作步长，在原理篇的正则化章节我们也讲到了，
#                加上了正则化项，我们的强学习器的迭代公式为fk(x)=fk−1(x)+νhk(x)。ν的取值范围为0<ν≤1。
#                对于同样的训练集拟合效果，较小的ν意味着我们需要更多的弱学习器的迭代次数。通常我们用步长和迭代最大次数一起来决定算法的拟合效果。
#                所以这两个参数n_estimators和learning_rate要一起调参。一般来说，可以从一个小一点的ν开始调参，默认是1。

# subsample: 即我們在原理篇的正則化章節講到的子採樣，取值爲(0,1]。
#             注意這裏的子採樣和隨機森林不一樣，隨機森林使用的是放回抽樣,
#             而這裏是不放回抽樣。如果取值爲1，則全部樣本都使用，等於沒有使用子採樣。
#             如果取值小於1，則只有一部分樣本會去做GBDT的決策樹擬合。選擇小於1的比例可以減少方差，
#             即防止過擬合，但是會增加樣本擬合的偏差，因此取值不能太低。推薦在[0.5, 0.8]之間，默認是1.0，即不使用子採樣。

# max_depth: 決策樹最大深度,默認可以不輸入，如果不輸入的話，默認值是3。一般來說，數據少或者特徵少的時候可以不管這個值。
#            如果模型樣本量多，特徵也多的情況下，推薦限制這個最大深度，具體的取值取決於數據的分佈。常用的可以取值10-100之間。

#max_features: 劃分時考慮的最大特徵數,可以使用很多種類型的值，默認是"None",意味着劃分時考慮所有的特徵數

#min_samples_split: 內部節點再劃分所需最小樣本數,這個值限制了子樹繼續劃分的條件，如果某節點的樣本數少於min_samples_split，
#                   則不會繼續再嘗試選擇最優特徵來進行劃分。 默認是2.如果樣本量不大，不需要管這個值。如果樣本量數量級非常大，則推薦增大這個值。

# min_samples_leaf: 葉子節點最少樣本數,這個值限制了葉子節點最少的樣本數，如果某葉子節點數目小於樣本數，則會和兄弟節點一起被剪枝。 
#             默認是1,可以輸入最少的樣本數的整數，或者最少樣本數佔樣本總數的百分比。如果樣本量不大，不需要管這個值。如果樣本量數量級非常大，則推薦增大這個值。

# bootstrap 和 bootstrap_features 控制著樣例和特徵的抽取是有放回還是無放回的。

# bootstrap:是統計學中的一種重採樣技術，可以簡單理解成是有放回地抽樣，默認是True,即採取有放回抽樣這種策略，這不就是bagging的思想麼。
#           oob_score:袋外估計(out-of-bag)，這個外是針對於bagging這個袋子而言的，我們知道，bagging採取的隨機抽樣的方式去建立樹模型，那麼那些未被抽取到的樣本集，
#           也就是未參與建立樹模型的數據集就是袋外數據集，我們就可以用這部分數據集去驗證模型效果，默認值爲False。

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
lr = LinearRegression()
gdbt = GradientBoostingRegressor(tol=0.1,subsample=0.37,n_estimators=200,max_features=20,max_depth=6,learning_rate=0.03)
rf = RandomForestRegressor(n_estimators=300,min_samples_split=9,min_samples_leaf=10,max_features='sqrt',max_depth=8,bootstrap=False)

In [8]:
model_lr = lr.fit(x_train,y_train)
lr_pred = model_lr.predict(x_test)
mse_lr = mean_squared_error(lr_pred,y_test,squared=False)
print(mse_lr)

88045.67690701502


In [9]:
model_rf = rf.fit(x_train,y_train)
rf_pred = model_rf.predict(x_test)
mse_rf = mean_squared_error(rf_pred,y_test,squared=False)
print(mse_rf)

86369.22179702045


In [12]:
#梯度提升機預測
model_gdbt = gdbt.fit(x_train, y_train)
gdbt_pred =gdbt.predict(x_test)
mse_gdbt = mean_squared_error(gdbt_pred, y_test,squared=False)
print(mse_gdbt)

96402.97065444443


In [13]:
#混合泛化預測
mse_sum = 1/mse_gdbt + 1/mse_rf +1/mse_lr
blending_pred = lr_pred*((1/mse_lr)/mse_sum)+rf_pred*((1/mse_rf)/mse_sum)+gdbt_pred*((1/mse_gdbt)/mse_sum)
mean_squared_error(blending_pred,y_test,squared=False)

73744.0717825205